# Best optimizer parameters:

## RSI_STOCH:

### BUY ~ 
- RSI: low/high bound: **35/65**, timeperiod: 14
- STOCH: low/high bound: **25/75**, fastk: 9, slowk: 3, slowd: 7

# Parameter optimization

In [34]:
import sys
import pandas as pd
import numpy as np
from optimizer import Optimizer
from os import environ, remove


# Set environment variable
environ["ENV"] = "optimize"

from bot.bot import SigBot
from config.config import ConfigFactory

# Get configs
configs = ConfigFactory.factory(environ).configs


pd.set_option('display.max_columns', 500)

ttype = 'buy'
pattern = ['STOCH', 'RSI']
opt_limit = 100
load = False

# optim_dict = {'RSI': {'timeperiod': [14, 16, 18, 20], 'low_bound': [20, 25, 30, 35]},
#               'STOCH': {'fastk_period': [4, 6, 8, 10], 'slowk_period': [2, 3, 4, 5],
#                         'slowd_period': [2, 3, 5, 7], 'low_bound': [10, 15, 20, 25]}}
optim_dict = {'RSI': {'timeperiod': [14], 'low_bound': [35]},
              'STOCH': {'fastk_period': [9], 'slowk_period': [3],
                        'slowd_period': [7], 'low_bound': [25]}}
opt = Optimizer(pattern, optim_dict, **configs)

stat = opt.optimize(pattern, ttype, opt_limit, load)
try:
    open(f'opt_{"_".join(pattern)}_{ttype}.pkl', 'w').close()
except FileNotFoundError:
    pass
stat.to_pickle(f'opt_{"_".join(pattern)}_{ttype}.pkl')

stat['pct_right_forecast_avg'] = stat[[f'pct_right_forecast_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat = stat.sort_values(['pct_right_forecast_avg', 'forecasts_num'], ascending=False)
stat.head(20)

Number of combinations is 1


  0%|          | 0/1 [00:00<?, ?it/s]

,pattern,RSI_timeperiod,RSI_low_bound,STOCH_fastk_period,STOCH_slowk_period,STOCH_slowd_period,STOCH_low_bound,pct_right_forecast_1,pct_right_forecast_2,pct_right_forecast_3,pct_right_forecast_4,pct_right_forecast_5,pct_right_forecast_6,pct_right_forecast_7,pct_right_forecast_8,pct_right_forecast_9,pct_right_forecast_10,pct_right_forecast_11,pct_right_forecast_12,pct_right_forecast_13,pct_right_forecast_14,pct_right_forecast_15,pct_right_forecast_16,pct_right_forecast_17,pct_right_forecast_18,pct_right_forecast_19,pct_right_forecast_20,pct_right_forecast_21,pct_right_forecast_22,pct_right_forecast_23,pct_right_forecast_24,pct_price_diff_1,pct_price_diff_2,pct_price_diff_3,pct_price_diff_4,pct_price_diff_5,pct_price_diff_6,pct_price_diff_7,pct_price_diff_8,pct_price_diff_9,pct_price_diff_10,pct_price_diff_11,pct_price_diff_12,pct_price_diff_13,pct_price_diff_14,pct_price_diff_15,pct_price_diff_16,pct_price_diff_17,pct_price_diff_18,pct_price_diff_19,pct_price_diff_20,pct_price_diff_21,pct_price_diff_22,pct_price_diff_23,pct_price_diff_24,forecasts_num,pct_right_forecast_avg
0,STOCH_RSI,14,35,90,3,7,25,73.91,78.26,78.26,82.61,82.61,78.26,69.57,69.57,65.22,65.22,56.52,56.52,56.52,60.87,65.22,73.91,82.61,78.26,82.61,73.91,78.26,78.26,78.26,78.26,0.13,0.22,0.26,0.3,0.41,0.3,0.18,0.26,0.09,0.14,0.09,0.09,0.18,0.18,0.19,0.26,0.57,0.52,0.48,0.4,0.35,0.35,0.6,0.66,23,72.645


# Save new config data to config file

In [2]:
import json
from optimizer import Optimizer

timeframe = "5m_1h"
ttype = 'buy'
pattern = []

with open(f'../config/config_{timeframe}.json', 'r') as f:
    configs = json.load(f)

optim_dict = {'RSI': {'timeperiod': 14, 'low_bound': 35},
              'STOCH': {'fastk_period': 9, 'slowk_period': 3,
                        'slowd_period': 7, 'low_bound': 25}}

opt = Optimizer(pattern, optim_dict, **configs)
confs = opt.save_configs(optim_dict, ttype)

with open(f'../config/config_{timeframe}.json', 'w') as f:
    json.dump(confs, f)